<a href="https://colab.research.google.com/github/ssgalitsky/PedalNetRT/blob/master/colab_TPU_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PyTorch Lightning Documentation https://pytorch-lightning.readthedocs.io/en/stable/

In [ ]:
#@title 1. clone  https://github.com/ssgalitsky/PedalNetRT.git
!git clone https://github.com/ssgalitsky/PedalNetRT.git

In [ ]:
#@title 2. install dependencies
!pip3 install torchvision
!pip3 install pytorch_lightning==0.7.3
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python3 pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
!pip3 install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.6-cp36-cp36m-linux_x86_64.whl

import torch
use_cuda=True
if use_cuda and torch.cuda.is_available():
  net.cuda()

#from google.colab import drive
#drive.mount('/content/drive')

import os
os.chdir('/content/PedalNetRT')
#os.getcwd()

#!ffmpeg -i "data/flute.wav"
#!ffmpeg -i "data/guitar.wav"
#!ffmpeg -i "data/guitar.wav" -acodec pcm_f32le -ar 44100 -ac 1 "data/guitar.wav"
#!ffmpeg -i "data/flute.wav" -acodec pcm_f32le -ar 44100 -ac 1 "data/flute.wav"

In [ ]:
#@title 3. prepare data
!python3 "prepare_data.py" "/content/PedalNetRT/data/guitar.wav" "/content/PedalNetRT/data/voice.wav"


In [ ]:
#@title 4. **train model**
import os
assert os.environ['COLAB_TPU_ADDR']
import pytorch_lightning
!python3 "train.py" --batch_size=32 --max_epochs=1500 --learning_rate=3e-3 --num_channels=5 





In [ ]:
#@title 4. train.py
import pytorch_lightning as pl
import argparse

from model import PedalNet


def main(args):
    model = PedalNet(args)
    trainer = pl.Trainer(
        #max_epochs=args.max_epochs, gpus=args.gpus, row_log_interval=100
        # The following line is for use with the Colab notebook when training on TPUs.
        # Comment out the above line and uncomment the below line to use.
        
         max_epochs=args.max_epochs, tpu_cores=args.tpu_cores, row_log_interval=100
    )
    trainer.fit(model)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--num_channels", type=int, default=5)
    parser.add_argument("--dilation_depth", type=int, default=10)
    parser.add_argument("--num_repeat", type=int, default=1)
    
    # filter_width=kernel_size
    parser.add_argument("--kernel_size", type=int, default=3)

    parser.add_argument("--batch_size", type=int, default=64)
    parser.add_argument("--learning_rate", type=float, default=3e-3)

    parser.add_argument("--max_epochs", type=int, default=100)
    parser.add_argument("--gpus", default="0")
    parser.add_argument("--tpu_cores", default="8")

    parser.add_argument("--data", default="data.pickle")
    args = parser.parse_args()
    main(args)

In [ ]:
#@title 5. convert torch model (ckpt) to plugin model (json) 
# The .ckpt model must be converted to a .json model to run in the plugin. Usage:
!python3 convert_pedalnet_to_wavnetva.py --model=your_trained_model.ckpt

In [ ]:
#@title 6. convert model from TPU to CPU format
import torch_xla
import torch_xla.core.xla_model as xm

from model import PedalNet
# Change path below to match model file
model = PedalNet.load_from_checkpoint('lightning_logs/version_0/checkpoints/epoch=1.ckpt')

# xm.save(model.state_dict(), 'tpu_to_cpu.ckpt') 
xm.save(model, 'tpu_to_cpu_model.ckpt') 
# issues loading model, see pytorch_lightning issues #2303 and #3044 (Might be completed by now)

# TODO: Add code to be able to load saved model sucessfully
